# **Image Classification**
- Cats & Dogs image classification from scratch (CNN)

In [ ]:
# Installing PyTorch Lightning

!pip install pytorch-lightning --quiet
!pip install torchmetrics

In [ ]:
import os
import torch
import torchmetrics
from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from PIL import Image
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [ ]:
# Download and unzip the Cats & Dogs dataset

!gdown --id 1Dvw0UpvItjig0JbnzbTgYKB-ibMrXdxk
!unzip -q dogs-vs-cats.zip
!unzip -q train.zip
!unzip -q test1.zip

In [ ]:
# Prepare dataset (Loading, Transforms, dataloaders)

# Returning the images and their annotations
class Dataset():
    def __init__(self, filelist, filepath, transform = None):
        self.filelist = filelist
        self.filepath = filepath
        self.transform = transform

    def __len__(self):
        return int(len(self.filelist))

    def __getitem__(self, index):
        imgpath = os.path.join(self.filepath, self.filelist[index])
        img = Image.open(imgpath)

        if "dog" in imgpath:
            label = 1
        else:
            label = 0

        if self.transform is not None:
            img = self.transform(img)

        return (img, label)


# Set directory paths for our files
train_dir = './train'
test_dir = './test1'

# Get files in our directories
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

# Create our transforms
transformations = transforms.Compose([transforms.Resize((60,60)),transforms.ToTensor()])

# Create our train and test dataset objects
train = Dataset(train_files, train_dir, transformations)
val = Dataset(test_files, test_dir, transformations)

# Split into our train and validation
train, val = torch.utils.data.random_split(train,[20000,5000])

train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = 32, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = val, batch_size = 32, shuffle=False)


In [ ]:
# Creating the model using Lightning

class LitModel(pl.LightningModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size
        self.conv1 = nn.Sequential(nn.Conv2d(3,16,3), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(16,32,3), nn.ReLU(), nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(nn.Conv2d(32,64,3), nn.ReLU(), nn.MaxPool2d(2,2))
        self.fc1 = nn.Sequential(nn.Flatten(), nn.Linear(64*5*5,256), nn.ReLU(), nn.Linear(256,128), nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128,2),)

    def train_dataloader(self):
        # transforms
        return torch.utils.data.DataLoader(dataset = train, batch_size = 32, shuffle=True)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(dataset = val, batch_size = 32, shuffle=False)

    def cross_entropy_loss(self, logits, labels):
      return F.nll_loss(logits, labels)

    def training_step(self, batch, batch_idx):
        data, label = batch
        output = self.forward(data)
        loss = nn.CrossEntropyLoss()(output,label)
        self.log('train_loss', loss)
        return {'loss': loss, 'log': self.log}

    def validation_step(self, batch, batch_idx):
        val_data, val_label = batch
        val_output = self.forward(val_data)
        val_loss = nn.CrossEntropyLoss()(val_output, val_label)
        self.log('val_loss', val_loss)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

    def forward(self, x):
        # in lightning, forward defines the prediction/inference actions
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return F.softmax(x,dim = 1)


In [ ]:
# Fit the model

model = LitModel(batch_size = 32)
trainer = pl.Trainer(max_epochs = 5)
# trainer = pl.Trainer(auto_scale_batch_size='binsearch')
trainer.fit(model)

In [ ]:
# Model checkpointing (Saving model at a certain point)

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

early_stop = EarlyStopping(
    monitor = 'val_loss', # One can choose loos or accuracy
    patience = 3, # The number of epochs before stopping
    strict = False,
    verbose = False,
    mode = 'min' # min if loss and max if accuracy
)


In [ ]:
# Setup Model Checkpoint (CallBacks)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='models/',
    filename='sample-catsvsdogs-{epoch:02d}-{val_loss:.2f}', # Saving the file with a certain name containing variables
    save_top_k=3,# We save the top 3 models
    mode='min',
)


In [ ]:
# We can even use some custom callbacks

from pytorch_lightning.callbacks import Callback


class MyPrintingCallback(Callback):

    def on_train_start(self, trainer, pl_module):
        print('Starting to init trainer!')

    def on_train_end(self, trainer, pl_module):
        print('trainer is init now')

    def on_train_end(self, trainer, pl_module):
        print('do something when training ends')

In [ ]:
# Fit the model

# init model
model = LitModel(batch_size = 32)

# Initialize a trainer
trainer = pl.Trainer(
    max_epochs=10,
    callbacks=[EarlyStopping('val_loss'), checkpoint_callback, MyPrintingCallback()]
)

trainer.fit(model)

In [ ]:
# Get path of best model

checkpoint_callback.best_model_path

In [ ]:
# loading the best checkpoints to model (Just inference)

pretrained_model = LitModel.load_from_checkpoint(batch_size = 32, learning_rate=0.001, checkpoint_path = checkpoint_callback.best_model_path)
pretrained_model = pretrained_model.to("cuda")
pretrained_model.eval()
pretrained_model.freeze()


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
# Visualizing sample predictions

samples, _ = next(iter(val_loader))
samples = samples.to('cuda')

fig = plt.figure(figsize=(12, 8))
fig.tight_layout()

output = pretrained_model(samples[:24])
pred = torch.argmax(output, dim=1)
pred = [p.item() for p in pred]
ad = {0:'cat', 1:'dog'}

for num, sample in enumerate(samples[:24]):
    plt.subplot(4,6,num+1)
    plt.title(ad[pred[num]])
    plt.axis('off')
    sample = sample.cpu().numpy()
    plt.imshow(np.transpose(sample, (1,2,0)))

# **Transfer Learning**
- Cats & Dogs image classification (CNN)
- Using PyTorch Lightning
- Fine-tuning a certail number of layers of an ImageNet model.

In [ ]:
# Install PyTorch Lightning and TorchMetrics

!pip install pytorch-lightning --quiet
!pip install torchmetrics

In [ ]:
import os
import torch
import torchmetrics
import torch.nn.functional as F

from torch import nn
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from PIL import Image
import torchvision.models as models


In [ ]:
# Download and unzip Cats & Dogs dataset

!gdown --id 1Dvw0UpvItjig0JbnzbTgYKB-ibMrXdxk
!unzip -q dogs-vs-cats.zip
!unzip -q train.zip
!unzip -q test1.zip

In [ ]:
# Preparing dataset

# The class for returning images with their annotations

class Dataset():
    def __init__(self, filelist, filepath, transform = None):
        self.filelist = filelist
        self.filepath = filepath
        self.transform = transform

    def __len__(self):
        return int(len(self.filelist))

    def __getitem__(self, index):
        imgpath = os.path.join(self.filepath, self.filelist[index])
        img = Image.open(imgpath)

        if "dog" in imgpath:
            label = 1
        else:
            label = 0

        if self.transform is not None:
            img = self.transform(img)

        return (img, label)


# Set directory paths for our files
train_dir = './train'
test_dir = './test1'

# Get files in our directories
train_files = os.listdir(train_dir)
test_files = os.listdir(test_dir)

# Applying transformations
transformations = transforms.Compose([transforms.Resize((60,60)),transforms.ToTensor()])

# Create our train and test dataset objects
train = Dataset(train_files, train_dir, transformations)
val = Dataset(test_files, test_dir, transformations)

# Splitting the dataset
train, val = torch.utils.data.random_split(train,[20000,5000])

# Setting up the the dataloaders
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size = 32, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset = val, batch_size = 32, shuffle=False)


In [ ]:
# Preparing an ImageNet based transfer learning model with PyTorch Lightning

class ImagenetTransferLearning(pl.LightningModule):
  def __init__(self):
    super().__init__()
    num_target_class = 2

    # Getting accuracy
    self.accuracy = torchmetrics.classification.Accuracy(task="binary")

    # Inint apretrained model
    backbone = models.resnet50(pretrained = True)
    layers = list(backbone.children())[:-1]
    self.feature_extractor = nn.Sequential(*layers)

    # Use the pretrained model
    num_filters = backbone.fc.in_features
    self.classifier = nn.Linear(num_filters, num_target_class)


  def forward(self, x):
    self.feature_extractor.eval()
    with torch.no_grad():
      representations = self.feature_extractor(x).flatten(1)
    x = self.classifier(representations)
    return F.softmax(x, dim = 1)


  def train_dataloader(self):
      return torch.utils.data.DataLoader(dataset = train, batch_size = 32, shuffle=True)


  def val_dataloader(self):
      return torch.utils.data.DataLoader(dataset = val, batch_size = 32, shuffle=False)


  def cross_entropy_loss(self, logits, labels):
      return F.nll_loss(logits, labels)


  def training_step(self, batch, batch_idx):
      data, label = batch
      output = self.forward(data)
      loss = nn.CrossEntropyLoss()(output, label)
      self.log('train_loss', loss)
      preds = torch.argmax(output, dim=1)
      self.log('train_acc_step', self.accuracy(preds, label))
      return {'loss': loss}


  def validation_step(self, batch, batch_idx):
      val_data, val_label = batch
      val_output = self.forward(val_data)
      val_loss = nn.CrossEntropyLoss()(val_output, val_label)
      self.log('val_loss', val_loss)
      val_preds = torch.argmax(val_output, dim=1)
      self.log('val_acc_step', self.accuracy(val_preds, val_label))


  def configure_optimizers(self):
      #optimizer = torch.optim.Adam(self.parameters(), lr=(self.learning_rate))
      optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
      return optimizer

  def on_train_epoch_end(self):
    accuracy = self.accuracy.compute()
    epoch = self.current_epoch
    self.log('train_acc_epoch', accuracy, on_epoch=True)
    print(f"Epoch Number {epoch}: Accuracy: {accuracy}")


In [ ]:
# Sarting the training process

model = ImagenetTransferLearning()
trainer = pl.Trainer(max_epochs = 2) # Set the number of epochs to your desired value
trainer.fit(model)